# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [44]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [45]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.82,73,17,5.04,PN,1712461804
1,1,cukai,4.2500,103.4167,30.97,60,48,2.74,MY,1712461804
2,2,edinburgh of the seven seas,-37.0676,-12.3116,18.80,95,99,9.75,SH,1712461804
3,3,kavaratti,10.5669,72.6420,29.07,70,98,1.27,IN,1712461804
4,4,steinbach,50.1667,8.5667,16.47,69,99,3.84,DE,1712461804


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [46]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [47]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"]>15) &
                                (city_data_df["Max Temp"]<30) &
                                (city_data_df["Humidity"]<40) &
                                (city_data_df["Wind Speed"]<10) &
                                (city_data_df["Cloudiness"]<20)
]

# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display sample data
city_data_df

/var/folders/tg/jpl0p2w93ndfvsd8ln2wc1pw0000gn/T/ipykernel_1218/1967552723.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,nasiriyah,31.0521,46.2610,26.02,24,12,5.55,IQ,1712461804
10,10,tamanrasset,22.7850,5.5228,22.95,12,0,9.77,DZ,1712461805
113,113,noria,32.5167,74.8000,20.77,15,0,0.79,IN,1712461812
148,148,basse santa su,13.3100,-14.2137,26.30,27,3,1.74,GM,1712461814
166,166,ubari,26.5921,12.7805,17.66,19,2,3.86,LY,1712461815
196,196,broken hill,-31.9500,141.4333,22.33,37,5,5.28,AU,1712461580
209,209,t'aebaek,37.1759,128.9889,17.87,28,6,2.02,KR,1712461819
272,272,aswan,24.0934,32.9070,19.88,25,9,5.57,EG,1712461668
310,310,muzaffargarh,30.0754,71.1921,24.01,10,0,3.08,PK,1712461828
316,316,taoudenni,22.6783,-3.9836,27.23,10,0,3.06,ML,1712461829


### Step 3: Create a new DataFrame called `hotel_df`.

In [48]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,nasiriyah,IQ,31.0521,46.2610,24,
10,tamanrasset,DZ,22.7850,5.5228,12,
113,noria,IN,32.5167,74.8000,15,
148,basse santa su,GM,13.3100,-14.2137,27,
166,ubari,LY,26.5921,12.7805,19,
196,broken hill,AU,-31.9500,141.4333,37,
209,t'aebaek,KR,37.1759,128.9889,28,
272,aswan,EG,24.0934,32.9070,25,
310,muzaffargarh,PK,30.0754,71.1921,10,
316,taoudenni,ML,22.6783,-3.9836,10,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [49]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey": geoapify_key,
          "categories": "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lng = hotel_df.loc[index,"Lng"]
    Lat = hotel_df.loc[index,"Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nasiriyah - nearest hotel: فندق حمورابي
tamanrasset - nearest hotel: فندق الأمان
noria - nearest hotel: No hotel found
basse santa su - nearest hotel: Basse Guesthouse
ubari - nearest hotel: فندق اوباري
broken hill - nearest hotel: Ibis Styles
t'aebaek - nearest hotel: 이스턴호텔
aswan - nearest hotel: Oscar Hotel
muzaffargarh - nearest hotel: Alhamdullilah Hotel
taoudenni - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
arar - nearest hotel: دار أوتيل Dar Otel
tindouf - nearest hotel: محمد بوسبي
aleg - nearest hotel: Hotel Oasis Aleg
murzuq - nearest hotel: فندق باريس - مرزق
shinas - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
7,nasiriyah,IQ,31.0521,46.2610,24,فندق حمورابي
10,tamanrasset,DZ,22.7850,5.5228,12,فندق الأمان
113,noria,IN,32.5167,74.8000,15,No hotel found
148,basse santa su,GM,13.3100,-14.2137,27,Basse Guesthouse
166,ubari,LY,26.5921,12.7805,19,فندق اوباري
196,broken hill,AU,-31.9500,141.4333,37,Ibis Styles
209,t'aebaek,KR,37.1759,128.9889,28,이스턴호텔
272,aswan,EG,24.0934,32.9070,25,Oscar Hotel
310,muzaffargarh,PK,30.0754,71.1921,10,Alhamdullilah Hotel
316,taoudenni,ML,22.6783,-3.9836,10,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [51]:
# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.5,
    hover_cols=["Country","Hotel Name"]
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)